# Sound Based Predictive Maintenance in SAP AI Core

In this notebook we will walk through the process of training and deploying a sound classification deep learning model in SAP AI Core.

## Create an AI API client instance

In [1]:
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client
import json

First of all, we need to create an AI API client instance, which will allow us to interact with our SAP AI Core tenant. Before executing the code, edit the [aic_bootcamp_service_key.json](../files/aic_bootcamp_service_key.json) file and enter the urls and keys of your tenant. 

In [2]:
# Your service key JSON file relative to your aicore instance
aic_service_key_path = '../files/aic_bootcamp_service_key.json'

# Loads the service key file
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

# Creating an AI API client instance
ai_api_client = AIAPIV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key['clientid'],
    client_secret = aic_service_key['clientsecret']
)

## 1. Create a Resource Group and Connect AWS S3 to SAP AI Core

Resource Groups represent a virtual collection of related resources within the scope of one SAP AI Core tenant. Let's create a specific resource group for our sound classification exercise. That resource group will then appear also on the AI Lauchpad UI. 

### List existing resource groups

In [4]:
ai_api_client.rest_client.delete(
    path=f"/admin/resourceGroups/sound"
)

{'id': 'sound', 'message': 'Resource Group deletion scheduled'}

In [5]:
ai_api_client.rest_client.get(
    path=f"/admin/resourceGroups"
)

{'count': 2,
 'resources': [{'created_at': '2022-04-23T12:12:00+00:00',
   'labels': [],
   'resource_group_id': 'defect-det-calabria',
   'status': 'PROVISIONED',
   'status_message': 'All onboarding steps are completed.',
   'tenant_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc',
   'zone_id': ''},
  {'created_at': '2022-04-20T03:21:15+00:00',
   'labels': [],
   'resource_group_id': 'default',
   'status': 'PROVISIONED',
   'status_message': 'All onboarding steps are completed.',
   'tenant_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc',
   'zone_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc'}]}

### Create a new resource group for sound classification task

In [6]:
resource_group='soundU0001' #!!! to be modified according to your username

In [7]:
ai_api_client.rest_client.post(
    path="/admin/resourceGroups",
    body={
        "resourceGroupId": resource_group # Name of your resource group
    }
)

{'resource_group_id': 'soundU0001',
 'tenant_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc',
 'zone_id': ''}

In [8]:
ai_api_client.rest_client.get(
    path=f"/admin/resourceGroups"
)

{'count': 3,
 'resources': [{'created_at': '2022-04-23T12:12:00+00:00',
   'labels': [],
   'resource_group_id': 'defect-det-calabria',
   'status': 'PROVISIONED',
   'status_message': 'All onboarding steps are completed.',
   'tenant_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc',
   'zone_id': ''},
  {'created_at': '2022-04-20T03:21:15+00:00',
   'labels': [],
   'resource_group_id': 'default',
   'status': 'PROVISIONED',
   'status_message': 'All onboarding steps are completed.',
   'tenant_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc',
   'zone_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc'},
  {'created_at': '2022-05-09T10:00:28+00:00',
   'labels': [],
   'resource_group_id': 'soundU0001',
   'status': 'PROVISIONED',
   'status_message': 'All onboarding steps are completed.',
   'tenant_id': 'dcf5ea6e-9b13-47d7-9486-747429386dfc',
   'zone_id': ''}]}

We are using AWS S3 Object Store as a cloud storage for our AI Core datasets and models. 
You can get AWS S3 Bucket from either of two ways:

- through SAP BTP Cockpit;

- through AWS. Refer AWS User Guide to S3.

For this exercise, we have already loaded our input dataset on a S3 Storage bucket for you. If you had to do this on your own, you would need to install AWS CLI and upload your dataset by means of a command like the following:
    aws s3 cp --recursive /local/path/to/data s3://your-bucket-id/sound/data/

Once the data are in the AWS S3, we need to register the Object Store to SAP AI Core resource group. In order to to that we prepare a json file with the needed AWS S3 credentials and we use a proper AI API in order to register it. Please enter your AWS S3 Object Store credentials in [s3_service_key.json](../files/s3_service_key.json) before executing the code.  

In [ ]:
# Loads your service key
s3_service_key_path = '../files/s3_service_key.json'

# Loads the service key file
with open(s3_service_key_path) as s3sk:
    s3_service_key = json.load(s3sk)

default_secret = {
    "name": "default", # Name of the connection
    "type": "S3",
    "endpoint": s3_service_key["host"],
    "bucket": s3_service_key["bucket"],
    "pathPrefix": "sound",
    "region": s3_service_key["region"],
    "data": {
        "AWS_ACCESS_KEY_ID": s3_service_key["access_key_id"],
        "AWS_SECRET_ACCESS_KEY": s3_service_key["secret_access_key"]
    }
}

# Call the api
ai_api_client.rest_client.post(
    path="/admin/objectStoreSecrets",
    body = default_secret, # defined above
    resource_group = resource_group
)

Let's make an API call to check if the artifact was corretly created. Object store secrets are stored under /admin/objectStoreSecrets and are linked to the resource group "sound", so those parameters have to be provided when creating and listing them. In case, with a similar call, secrets can be also deleted.

In [ ]:
ai_api_client.rest_client.get(
    path="/admin/objectStoreSecrets",
    resource_group = resource_group
)

## 2. Train Sound Classification model

### 2.1 Create training docker image

In order to execute a training pipeline in AI Core, you need to dockerize your training code and push the docker image to your docker registry. You should have already linked your docker registrey to SAP AI Core during the previous exercises

### 2.2 Register training workflow as an application

After having prepared the docker image, you need to create a training workflow in the github repository associated to our AI Core instance. The yaml file needs to be uploaded in a dedicated folder of the github repo. You can then register your application using the AI API as follows. Before executing the code, open files/git_setup.json and double check everything is configured correctly. 

#### List available repositories

In [ ]:
ai_api_client.rest_client.get(
    path="/admin/repositories"
)

#### List available applications

In [ ]:
ai_api_client.rest_client.get(
    path="/admin/applications"
)

#### Add a new application for sound classification model

In [ ]:
# Loads your git_setup.json
with open('../files/git_setup.json') as gs:
    setup_json = json.load(gs)
    
# Registers the directory as app
app_json = setup_json["app"]
response = ai_api_client.rest_client.post(
    path="/admin/applications",
    body={
        "applicationName": app_json["applicationName"],
        "repositoryUrl": app_json["repositoryUrl"],
        "revision": app_json["revision"],
        "path": app_json["path"]
    }
)

In [ ]:
ai_api_client.rest_client.get(
    path="/admin/applications"
)

It is always a good practice to check the synchronization of the workflows (registered as apps), that are read from the registered GitHub repository. 
Please, keep in mind that the synchronization is triggered by any change to the yaml files pushed to GitHub and that AI Core checks every 3 minutes for new files or changes.

In [ ]:
app_name = "aicore-sound"

ai_api_client.rest_client.get(
    path=f"/admin/applications/{app_name}/status"
)

### 2.3 - Choose a scenario  

In [ ]:
import sys, os
import json
import requests
import base64
import time
import yaml
from IPython.display import clear_output
from pprint import pprint

from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.status import Status
from ai_api_client_sdk.models.target_status import TargetStatus
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding

Let's check which scenarios are available for your sound resource group. Previously we have defined some scenarios (through the provided workflow yaml files) for our resource group. We can check if they are present:

In [ ]:
scenario_resp = ai_api_v2_client.scenario.query(resource_group)

print("Scenarios")
print("---" * 20)
for idx, scenario in enumerate(scenario_resp.resources):
    print(f"Scenario ID {idx + 1} -> [{scenario.id}]")

In order to be used, training data need to be registered as artifact in AI Core. This means specify some data parameters and associate them to an existing scenario. Below an example on how to perform this registration in the context of our exercise.


Please, note that ai://default/data is the path to the data in S3 and "default" is specified in the secret object store registration. If you have specified also a pathPrefix, that will be already include in "default".


The scenario id is assigned as the one specified into the training workflows (we read it from the local file, and it should match the one read by AICore from the GitHub repository). At the end of the registrazioni step, a unique id is assigned to the artifact. Once registered the artifact will appear in the "Dataset" section in AI Launchpad.

NB: This step needs to be executed only when registering a new dataset, or something has changed in the current dataset.

In [ ]:
# NO CHANGES REQUIRED BELOW
#
# Load training_workflow.yaml
training_workflow_file = '../workflows_sound/training_workflow.yaml'
with open(training_workflow_file) as twf:
    training_workflow = yaml.safe_load(twf)
#
# Load scenario id from train_workflow.yaml
scenario_id = training_workflow['metadata']['labels']['scenarios.ai.sap.com/id']
#
# Set the artifact configuration
artifact = {
        "name": "sound-data", # Modifiable name
        "kind": Artifact.Kind.DATASET,
        "url": "ai://default/data", #default is 
        "description": "Cutting machine sound clips for defect detection",
        "scenario_id": scenario_id
    }
# Store the artifact response to retrieve the id for the training configuration
artifact_resp = ai_api_v2_client.artifact.create(**artifact)
print(f"Artifacts registered for {scenario_id} scenario!")
pprint(vars(artifact_resp))
#
# Checks if the message contains expected string
assert artifact_resp.message == 'Artifact acknowledged'

In order to train our model, we need to create a training configuration in AICore. A configuration convey several information about the training:

- the name of the configuration itself;
- the name that will indetify the input data inside AICore (it is assigned in the workflow);
- the name that will identify the training executable in AICore (it is assigned in the workflow);
- the id of the related scenario

At the end of the configuration step, a unique id is assigned to the configuration. Once created the configuration will appear in the "Configuration" section in AI Launchpad.

NB: Also in this case, the configuration is created one time, unless something has changed or you need to execute another training on a different dataset.

In [ ]:
input_artifact_name = training_workflow['spec']['templates'][0]['inputs']['artifacts'][0]['name']
executable_name = training_workflow['metadata']['name']

artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": artifact_resp.id
}

train_configuration = {
    "name": "sound-training-configuration",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

# store the configuration response to access the id to create an execution
train_config_resp = ai_api_v2_client.configuration.create(**train_configuration)
pprint(vars(train_config_resp))

assert train_config_resp.message == 'Configuration created'

print("Configuration created for running the training")

Once the configuration is created, we are ready to creare an execution that will be immediatly launched:

In [ ]:
execution_resp = ai_api_v2_client.execution.create(train_config_resp.id)
pprint(vars(execution_resp))

The following code provide information about the training status. The training status can be also checked in AI Launchpad under the "Execution" sections.

Please, note that the execution is performed inside the docker image previosly prepared and loaded in the Docker repository. The information needed to access the Docker training image are specified inside the training workflow file loaded in GitHub which AICore is synchronized with.

Once the training is complete, a model is saved and it becomes available under the AI Launchpad "Models" section for deploying a serving artifact.

In [ ]:
status = None
while status != Status.COMPLETED and status != Status.DEAD:
    # Sleep for 5 secs to avoid overwhelming the API with requests
    time.sleep(5)
    # Clear outputs to reduce clutter
    clear_output(wait=True)

    execution = ai_api_v2_client.execution.get(execution_resp.id)
    status = execution.status
    print('...... execution status ......', flush=True)
    print(f"Training status: {execution.status}")
    print(f"Training status details: {execution.status_details}")


if execution.status == Status.COMPLETED:
    print(f"Training complete for execution [{execution_resp.id}]!")
    output_artifact = execution.output_artifacts[0]
    output = {
        "id": output_artifact.id,
        "name": output_artifact.name,
        "url": output_artifact.url
    }
    with open('training_output.json', 'w') as fp:
        json.dump(output, fp)

# Metrics and performance

A common need is to check the performance of the training. To that it is possible to define some metrics in the training code and register them inside AiCore. This will made them available once the training is complete.

Below some example about how to retrieve the metrics from AICore.

In [ ]:
import ast
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
filter_string = "executionId eq '" + execution_resp.id + "'"
metric_resp = ai_api_v2_client.metrics.query(execution_ids=execution_resp.id)

for m in metric_resp.resources:
    for metric in m.metrics:
        print(metric.name)
        print(metric.value)

In [ ]:
all_metrics = []
for m in metric_resp.resources:
    for custom_info in m.custom_info:
        #print(custom_info.name)
        #print(custom_info.value)
        all_metrics.append(custom_info.value)

In [ ]:
training_metrics = ast.literal_eval(all_metrics[0])
confusion_matrix = ast.literal_eval(all_metrics[1])
#print(training_metrics[0].get("loss"))
#print(confusion_matrix[0].get("actual label - 0"))
#print(confusion_matrix[1].get("actual label - 1"))

a = confusion_matrix[0].get("actual label - 0")
b = confusion_matrix[1].get("actual label - 1")

a = [int(x) for x in re.split('[^0-9]', a) if x]
b = [int(x) for x in re.split('[^0-9]', b) if x]
#print(a)
#print(b)

cnf_matrix = np.array([a, b])
print(cnf_matrix)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

a = ast.literal_eval(training_metrics[0].get("loss"))
b = ast.literal_eval(training_metrics[1].get("val_loss"))
c = ast.literal_eval(training_metrics[2].get("accuracy"))
d = ast.literal_eval(training_metrics[3].get("val_accuracy"))

axs[0].plot(a)
axs[0].plot(b)
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(c)
axs[1].plot(d)
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
import itertools

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=["normaly","anomaly"], normalize=True,
                      title='Confusion matrix with normalization')

# Deploy ML Model

The model deployment is aimed at the creation and the deployment of a serving API (or artifact) that can be called to perform the inference step on some data.

To do that, first one needs to provide a piece of code where it is specified the behaviour of the artifact and a proper environment where the serving artifact can run. Assuming the code has been developed, one has to create a proper Docker image for running the application.

In order to prepare the docker image with the proper content, a Docker file and a requirement.txt file have to be prepared. Then the image needs to be built and uploaded with the following commands.

- docker build -t YOUR_DOCKER_REPO/sound-serve:0.0.1 .
- docker push docker.io/YOUR_DOCKER_REPO/sound-serve:0.0.1

In [ ]:
serving_workflow_file = '../workflows_sound/serving_workflow_tutorial.yaml'
with open(serving_workflow_file) as swf:
    serving_workflow = yaml.safe_load(swf)

scenario_id = serving_workflow['metadata']['labels']['scenarios.ai.sap.com/id']
input_artifact_name = serving_workflow['spec']['inputs']['artifacts'][0]['name']
executable_name = serving_workflow['metadata']['name']

training_output = 'training_output.json'
with open(training_output) as to:
    serving_input = json.load(to)

In [ ]:
artifact_binding = {
    "key": input_artifact_name,
    "artifact_id": serving_input["id"]
}

serve_configuration = {
    "name": "dev-tutorial-serving-configuration",
    "scenario_id": scenario_id,
    "executable_id": executable_name,
    "parameter_bindings": [],
    "input_artifact_bindings": [ InputArtifactBinding(**artifact_binding) ]
}

serve_config_resp = ai_api_v2_client.configuration.create(**serve_configuration)

assert serve_config_resp.message == 'Configuration created'

pprint(vars(serve_config_resp))
print("configuration for serving the model created")

In [ ]:
deployment_resp = ai_api_v2_client.deployment.create(serve_config_resp.id)
pprint(vars(deployment_resp))

In [ ]:
# Poll deployment status
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print('...... deployment status ......', flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")

# Allow some time for deployment URL to get ready
time.sleep(10)

# Using deployed ML model

Let's define the local path to the dataset:

In [ ]:
import glob

path_normal_sounds = glob.glob("../data/normal/*")
path_abnormal_sounds = glob.glob("../data/abnormal/*")
#print(path_normal_sounds)
#print(path_abnormal_sounds)

Let's visualize some audio clips:

In [ ]:
import librosa
import librosa.display

def acoustic_feature_computation(clip,sr):
    #scale, sr = librosa.load(clip)
    scale=clip
    mel_spectrogram = librosa.feature.melspectrogram(
        y=scale, 
        sr=sr, 
        hop_length=512,
        n_mels=64,
        fmax=sr/2
    )
    log_mel = librosa.power_to_db(mel_spectrogram)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    MFCCs=librosa.feature.mfcc(y=scale, sr=sr, n_mfcc=40, fmax=sr/2)
    acoustic_features=np.concatenate((MFCCs,log_mel_spectrogram), axis=0)
    return acoustic_features

In [ ]:
scale_abn, sr_abn = librosa.load(path_abnormal_sounds[0])
scale_norm, sr_norm = librosa.load(path_normal_sounds[0])
#print(scale_abn, scale_norm)

First let's visualize the two examples:

In [ ]:
acoustic_features_abn = acoustic_feature_computation(scale_abn,sr_abn)
acoustic_features_norm = acoustic_feature_computation(scale_norm,sr_norm)
print(acoustic_features_abn.shape)
print(acoustic_features_norm.shape)

Abnormal sound clip:

In [ ]:
plt.pcolor(acoustic_features_abn)
plt.show()

Normal sound clip:

In [ ]:
plt.pcolor(acoustic_features_norm)
plt.show()

In order to perform the inference step, let's transform one of the sound clips into a string (this will constitute  the body of the API call):

In [ ]:
from base64 import b64encode
import base64
import io
from json import dumps

ENCODING = 'utf-8'

# first: reading the binary stuff
# note the 'rb' flag
# result: bytes
with open(path_abnormal_sounds[0], 'rb') as open_file:
    byte_content = open_file.read()

# second: base64 encode read data
# result: bytes (again)
base64_bytes = b64encode(byte_content)

# third: decode these bytes to text
# result: string (in utf-8)
base64_string = base64_bytes.decode(ENCODING)

# optional: doing stuff with the data
# result here: some dict
raw_data = {"sound": base64_string}

In [ ]:
f = open("sounds.txt", "w")
f.write(base64_string)
f.close()
#print(base64_string)

Let's make a call to the API for retrieving the prediction:

In [ ]:
endpoint = f"{deployment.deployment_url}/v1/models/soundmodel:predict"
print(endpoint)

headers = {"Authorization": ai_api_v2_client.rest_client.get_token(),
           'ai-resource-group': resource_group,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=raw_data)

print('Inference result:', response.json())
#pprint(vars(response))

# Stop deployed model

In [ ]:
delete_resp = ai_api_v2_client.deployment.modify(deployment_resp.id,
                                                 target_status=TargetStatus.STOPPED)
status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_api_v2_client.deployment.get(deployment_resp.id)
    status = deployment.status
    print('...... killing deployment ......', flush=True)
    print(f"Deployment status: {deployment.status}")